### Imports

In [10]:
# Libraries
import os
from importlib import reload

import tensorflow as tf
import keras
import sklearn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Scripts
from helpers import data, runners, utils, generators

In [2]:
!pip show tensorflow # Should be v2.10

Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [3]:
!pip show keras # Should be v2.10

Name: keras


Version: 2.10.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: c:\users\sonar\anaconda3v2\envs\acml_2024\lib\site-packages
Requires: 
Required-by: keras-tuner, tensorflow


In [4]:
reload(runners)
reload(data)
reload(utils)

<module 'helpers.utils' from 'c:\\Users\\sonar\\Desktop\\thesis-ml\\ml\\main\\helpers\\utils.py'>

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
utils.check_gpu_use() 

GPU DETECTED ✓
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## `main`

In [ ]:
NUM_EPOCHS = 2
DATA_BATCH_SIZE = 16
GPU_BATCH_SIZE = 4

### Load data

In [7]:
pairs_df = pd.read_csv("../data/DATASET_CSVS/deepship_pairs_diff_recording.csv")

print(pairs_df.shape)

(12880, 5)


In [8]:
# ADD RELATIVE PATH

PATH_TO_ROOT = "..\\data\\"
pairs_df["file_path_1"] = pairs_df["file_path_1"].apply(
    lambda x: PATH_TO_ROOT + x
)
pairs_df["file_path_2"] = pairs_df["file_path_2"].apply(
    lambda x: PATH_TO_ROOT + x
)

pairs_df.head(2)

,ship_name,file_path_1,date_seg_1,file_path_2,date_seg_2
0,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_1,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_51
1,ALFRED_N,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170326_2,..\data\deepship_baseline_mat\Tanker\ALFRED_N-...,20170403_48


In [14]:
# Get train-validate-test splits

train_df, test_df = sklearn.model_selection.train_test_split(
    pairs_df,
    test_size=0.2,
    random_state=100,
    shuffle=True
)

val_df, test_df = sklearn.model_selection.train_test_split(
    test_df,
    test_size=0.5,
    random_state=100,
    shuffle=True
)

In [15]:
# Initialise generators
train_gen = generators.N2NGenerator(
    train_df,
    'mat',
    'Pexp',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

val_gen = generators.N2NGenerator(
    val_df,
    'mat',
    'Pexp',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

test_gen = generators.N2NGenerator(
    test_df,
    'mat',
    'Pexp',
    batch_size=DATA_BATCH_SIZE,
    shuffle=True,
    conv_channel=True
)

### Initialise model

In [16]:
def psnr(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

In [17]:
# model = keras.saving.load_model('models/saved/n2n_finetuned_25112024/n2n_finetuned.keras')

from models import irfan_2020, unet, unetpro

model = irfan_2020.get_irfan_model(input_shape=(192, 192, 1))
# model = unet.get_unet_model(input_size=(192, 192, 1))

# model = unetpro.get_unetpro_model(
#     input_size=(192, 192, 1), 
#     classes=1, # Use 1 for denoising
#     dropout=0.2
# ) 

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.99, epsilon=1e-8),
    loss=keras.losses.MeanSquaredError(),
    metrics=[psnr]
)

model.summary()

Model: "irfan_2020"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 192, 192, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 192, 64)      640       
                                                                 
 activation (Activation)     (None, 192, 192, 64)      0         
                                                                 
 batch_normalization (BatchN  (None, 192, 192, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 96, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 64)        3

### Train model

In [ ]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="models/saved/diff_spec_denoiser_05122024/irfan/{epoch:02d}.keras",
    monitor='val_psnr',
    mode='max',
    save_best_only=True
)

csv_logger = keras.callbacks.CSVLogger(
    filename="models/saved/diff_spec_denoiser_05122024/irfan/training.log",
    separator=",",
    append=False
)

backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir="models/saved/diff_spec_denoiser_05122024/irfan/tmp"
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    batch_size=GPU_BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[model_checkpoint_callback, csv_logger, backup_callback]
    verbose=2
)

In [ ]:
evals = model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

In [ ]:
# model.save('models/saved/unet_denoiser_11112024/unet_denoiser.keras')
model.save('models/saved/diff_spec_denoiser_05122024/irfan/irfan_denoiser.keras')

### Evaluation, metrics, and visualising denoised output

In [ ]:
reload(utils)
fig = utils.get_psnr_and_loss_curves(history, together=True)
fig.savefig("models/saved/diff_spec_denoiser_05122024/irfan/img/psnr_loss_curves.pdf")

In [ ]:
# Get a few samples from the test generator for visualisation and metrics calculation
num_samples = 5 

sample_X, sample_y = next(iter(test_gen))
sample_inputs = sample_X[:num_samples]  # Extract the first `num_samples` examples from the batch
sample_outputs = sample_y[:num_samples]  # Extract the first `num_samples` examples from the batch

# Predict the denoised outputs
denoised_outputs = model.predict(sample_inputs)

In [ ]:
# Get evaluation metrics
mse_score = tf.keras.losses.MeanSquaredError()(sample_inputs, denoised_outputs).numpy()
psnr_score = psnr(sample_inputs, denoised_outputs).numpy()

print(f"MSE on sample batch: {mse_score}")
print(f"PSNR on sample batch: {psnr_score}")

In [ ]:
# Check if all outputs are normalised
for i, output in enumerate(denoised_outputs):
    print(f"Sample {i+1} - Min: {np.min(output)}; Max: {np.max(output)}")

In [ ]:
# Plot the input, output, and target spectrograms for each sample
for i in range(num_samples):
    plt.figure(figsize=(12, 4))
    
    # INPUT
    plt.subplot(1, 3, 1)
    plt.imshow(sample_inputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Input")
    # plt.colorbar()

    # DENOISED OUTPUT
    plt.subplot(1, 3, 2)
    plt.imshow(denoised_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Denoised Output")
    # plt.colorbar()

    # TARGET
    plt.subplot(1, 3, 3)
    plt.imshow(sample_outputs[i].squeeze(), cmap="viridis", aspect="auto")
    plt.title("Target")
    # plt.colorbar()
    
    plt.tight_layout()
    plt.show()